In [41]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [42]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

# Specify the file path
file_path = "../Data/prosperLoanData.csv"

In [43]:
# Read the CSV file into a Pandas DataFrame
Uncleaned_df = pd.read_csv(file_path)

# Review the DataFrame

Uncleaned_df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [44]:
#Columns we need.
specific_columns = ['Term','LoanStatus','ProsperRating (numeric)','Occupation','EmploymentStatus','IsBorrowerHomeowner','IncomeRange','CreditScoreRangeLower','CreditScoreRangeUpper','RevolvingCreditBalance',
'BorrowerRate','EmploymentStatusDuration','LoanMonthsSinceOrigination','MonthlyLoanPayment','LoanOriginalAmount']  
# List of columns you want to check



In [45]:
specific_columns_uncleaned_df = Uncleaned_df[specific_columns]
specific_columns_uncleaned_df.head(10)

,Term,LoanStatus,ProsperRating (numeric),Occupation,EmploymentStatus,IsBorrowerHomeowner,IncomeRange,CreditScoreRangeLower,CreditScoreRangeUpper,RevolvingCreditBalance,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,LoanOriginalAmount
0,36,Completed,NaN,Other,Self-employed,True,"$25,000-49,999",640.0,659.0,0.0,0.1580,2.0,78,330.43,9425
1,36,Current,6.0,Professional,Employed,False,"$50,000-74,999",680.0,699.0,3989.0,0.0920,44.0,0,318.93,10000
2,36,Completed,NaN,Other,Not available,False,Not displayed,480.0,499.0,NaN,0.2750,NaN,86,123.32,3001
3,36,Current,6.0,Skilled Labor,Employed,True,"$25,000-49,999",800.0,819.0,1444.0,0.0974,113.0,16,321.45,10000
4,36,Current,3.0,Executive,Employed,True,"$100,000+",680.0,699.0,6193.0,0.2085,44.0,6,563.97,15000
5,60,Current,5.0,Professional,Employed,True,"$100,000+",740.0,759.0,62999.0,0.1314,82.0,3,342.37,15000
6,36,Current,2.0,Sales - Retail,Employed,False,"$25,000-49,999",680.0,699.0,5812.0,0.2712,172.0,11,122.67,3000
7,36,Current,4.0,Laborer,Employed,False,"$25,000-49,999",700.0,719.0,1260.0,0.2019,103.0,10,372.60,10000
8,36,Current,7.0,Food Service,Employed,True,"$25,000-49,999",820.0,839.0,9906.0,0.0629,269.0,3,305.54,10000
9,36,Current,7.0,Food Service,Employed,True,"$25,000-49,999",820.0,839.0,9906.0,0.0629,269.0,3,305.54,10000


In [46]:
specific_columns_uncleaned_df.dtypes

Term                            int64
LoanStatus                     object
ProsperRating (numeric)       float64
Occupation                     object
EmploymentStatus               object
IsBorrowerHomeowner              bool
IncomeRange                    object
CreditScoreRangeLower         float64
CreditScoreRangeUpper         float64
RevolvingCreditBalance        float64
BorrowerRate                  float64
EmploymentStatusDuration      float64
LoanMonthsSinceOrigination      int64
MonthlyLoanPayment            float64
LoanOriginalAmount              int64
dtype: object

In [47]:
columns_to_scale = ['Term',
                                   'ProsperRating (numeric)',
                                   'CreditScoreRangeLower',
                                   'CreditScoreRangeUpper',
                                   'RevolvingCreditBalance',
                                   'BorrowerRate',
                                   'EmploymentStatusDuration',
                                   'LoanMonthsSinceOrigination',
                                   'MonthlyLoanPayment',
                                   'LoanOriginalAmount'
]

In [48]:
specific_columns_uncleaned_df = specific_columns_uncleaned_df.dropna(subset=columns_to_scale)


In [49]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(specific_columns_uncleaned_df[columns_to_scale])

In [50]:
scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)

In [51]:
specific_columns_uncleaned_df[columns_to_scale] = scaled_df

In [52]:
clean_df = specific_columns_uncleaned_df
clean_df.head()

,Term,LoanStatus,ProsperRating (numeric),Occupation,EmploymentStatus,IsBorrowerHomeowner,IncomeRange,CreditScoreRangeLower,CreditScoreRangeUpper,RevolvingCreditBalance,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,LoanOriginalAmount
1,-0.557261,Current,1.152074,Professional,Employed,False,"$50,000-74,999",2.136159,2.136159,-0.526144,-1.321398,0.102440,-0.001558,0.158056,0.145693
3,1.504434,Current,0.554423,Skilled Labor,Employed,True,"$25,000-49,999",0.862158,0.862158,1.437989,-0.865828,-0.216890,-0.937078,0.270123,0.940903
4,-0.557261,Current,-1.238529,Executive,Employed,True,"$100,000+",-0.411842,-0.411842,-0.386768,1.007371,0.710197,-0.361374,-0.906786,-0.967602
5,-0.557261,Current,-0.043228,Professional,Employed,True,"$100,000+",0.012825,0.012825,-0.532016,0.078811,-0.000570,-0.433337,0.432061,0.145693
6,-0.557261,Current,1.749725,Sales - Retail,Employed,False,"$25,000-49,999",2.560825,2.560825,-0.256134,-1.783668,1.709390,-0.937078,0.072828,0.145693


In [57]:
clean_df.dtypes


Term                          float64
LoanStatus                     object
ProsperRating (numeric)       float64
Occupation                     object
EmploymentStatus               object
IsBorrowerHomeowner              bool
IncomeRange                    object
CreditScoreRangeLower         float64
CreditScoreRangeUpper         float64
RevolvingCreditBalance        float64
BorrowerRate                  float64
EmploymentStatusDuration      float64
LoanMonthsSinceOrigination    float64
MonthlyLoanPayment            float64
LoanOriginalAmount            float64
dtype: object

In [64]:
dummies = pd.get_dummies(clean_df["IncomeRange"],dtype=int)
dummies.head()

,$0,"$1-24,999","$100,000+","$25,000-49,999","$50,000-74,999","$75,000-99,999",Not employed
1,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0
4,0,0,1,0,0,0,0
5,0,0,1,0,0,0,0
6,0,0,0,1,0,0,0
